### Problem 1
Read the noisy text data from the datafile [typos20.data](https://www.mbolonkin.info/teaching/cs170-2024/notebooks/typos20.data).
- Estimate output probabilities
- Estimate transition probabilities

Use Laplace smoothing.

Implement Viterbi algorithm for HMM. Find the most likely original text.

In [1]:
import numpy as np

In [2]:
orig, typo = [], []
i2c, c2i = {}, {}
i = 0
with open('typos20.data') as fin:
    for line in fin:
        toks = line.strip().split(' ')
        if len(toks) < 2:
            continue
        orig.append(toks[0])
        typo.append(toks[1])
        if toks[0] not in c2i:
            c2i[toks[0]] = i
            i2c[i] = toks[0]
            i += 1
        if toks[1] not in c2i:
            c2i[toks[1]] = i
            i2c[i] = toks[1]
            i += 1

In [3]:
eps = 1e-10
n = len(i2c)
A = np.zeros((n, n))
for t in range(len(orig)-1):
    i = c2i[orig[t]]
    j = c2i[orig[t+1]]
    A[i,j] += 1
A = (A + eps)/ (A.sum(axis=1) + n*eps)

pi = np.zeros(n)
B = np.zeros((n, n))
for t in range(len(orig)):
    i = c2i[orig[t]]
    j = c2i[typo[t]]
    B[i,j] += 1
    pi[i] += 1
B = (B + eps) / (B.sum(axis=1) + n*eps)
pi = pi/(pi.sum() + 1)

In [32]:
T = 10000
X = np.zeros((n, T))
Y = np.zeros((n, T))

In [33]:
for i in range(n):
    X[i,0] = np.log(pi[i]) + np.log(B[i, c2i[typo[0]]])
    
for t in range(1, T):
    for i in range(n):
        p = X[0, t-1] + np.log(A[0, i]) +  np.log(B[i, c2i[typo[t]]])
        idx = 0
        for j in range(1, n):
            p_ji = X[j, t-1] + np.log(A[j, i]) +  np.log(B[i, c2i[typo[t]]])
            if p_ji > p:
                p = p_ji
                idx = j
        X[i, t] = p
        Y[i, t] = idx

In [34]:
restored = ['a']*T
idx = np.argmax(X[:,-1])

for t in range(T):
    restored[T-t-1] = i2c[idx]
    idx = int(Y[idx, T-t-1])

In [35]:
err1 = 0
for i in range(T):
    if orig[i] != typo[i]:
        err1 += 1
print(err1/len(orig))

err2 = 0
for i in range(T):
    if orig[i] != restored[i]:
        err2 += 1
print(err2/len(orig))

0.008743730805770972
0.006396235169351097


### Problem 2
Solve the above problem but considering bi-grams (pairs of characters) instead of single characters. For example, in the word "PROBLEM" we have bigrams {"PR", "RO", "BL", "LE", "EM"}. Compare the recovery accuracy to the original error.

### Problem 3
Use the typo data from the filetry estimate the probabi